In [6]:
import os
import requests

from dotenv import load_dotenv
from flask import Flask, jsonify

from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:3000"}})

load_dotenv()

GITHUB_REPO = "breba-apps/TempRepo"  # Change to your repository
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")  # Use an environment variable for security

HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"} if GITHUB_TOKEN else {}


def get_most_prolific_author():
    url = f"https://api.github.com/repos/{GITHUB_REPO}/issues?state=all&per_page=100"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code != 200:
        return None, 0

    issues = response.json()
    author_count = {}

    for issue in issues:
        author = issue["user"]["login"]
        author_count[author] = author_count.get(author, 0) + 1

    # Find the author with the most issues
    most_prolific = max(author_count, key=author_count.get, default=None)
    return most_prolific, author_count.get(most_prolific, 0)


def count_author_comments(author):
    url = f"https://api.github.com/repos/{GITHUB_REPO}/issues/comments?per_page=100"
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        return 0

    comments = response.json()
    return sum(1 for comment in comments if comment["user"]["login"] == author)


@app.route("/data")
def get_github_data():
    author, issue_count = get_most_prolific_author()
    
    if not author:
        return jsonify({"error": "Failed to fetch GitHub data"}), 500

    comment_count = count_author_comments(author)

    result = {"data": [{"name": author, "issue_count": issue_count, "comment_count": comment_count}]}
    return jsonify(result)


In [ ]:
app.run(debug=False, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [28/Feb/2025 19:25:19] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 19:28:28] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 19:29:14] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2025 19:29:30] "GET /data HTTP/1.1" 200 -
